In [12]:
import pandas as pd
import pandas_ta as ta
import numpy as np
import plotly.graph_objects as go
from tqdm import tqdm

In [15]:
df = pd.read_csv("raw_btcusd.csv")
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df['RSI'] = ta.rsi(df.close, length=12)
df.tail()

df=df[:20000]

In [16]:
df

,time,open,high,low,close,tick_volume,RSI
0,2017-03-28 03:00:00,1041.319,1041.320,1041.220,1041.222,68,NaN
1,2017-03-28 04:00:00,1041.222,1042.778,1041.220,1042.000,185,NaN
2,2017-03-28 05:00:00,1043.000,1043.000,1042.007,1042.007,32,NaN
3,2017-03-28 06:00:00,1042.989,1042.989,1042.000,1042.003,50,NaN
4,2017-03-28 07:00:00,1043.000,1043.000,1041.504,1042.007,162,NaN
...,...,...,...,...,...,...,...
19995,2019-09-03 14:00:00,10658.042,10752.053,10621.942,10623.112,1657,77.778269
19996,2019-09-03 15:00:00,10623.112,10679.499,10573.311,10679.499,1500,79.928256
19997,2019-09-03 16:00:00,10679.499,10686.485,10635.587,10636.585,1128,73.985180
19998,2019-09-03 17:00:00,10636.585,10669.020,10594.170,10594.170,658,68.493919


# Support and Resitance logic

In [17]:
wick_threshold = 0.0001
def support(df1, l, n1, n2): #n1 n2 before and after candle l
    if ( df1.low[l-n1:l].min() < df1.low[l] or
        df1.low[l+1:l+n2+1].min() < df1.low[l] ):
        return 0

    candle_body = abs(df1.open[l]-df1.close[l])
    lower_wick = min(df1.open[l], df1.close[l])-df1.low[l]
    if (lower_wick > candle_body) and (lower_wick > wick_threshold): 
        return 1
    
    return 0

def resistance(df1, l, n1, n2): #n1 n2 before and after candle l
    if ( df1.high[l-n1:l].max() > df1.high[l] or
       df1.high[l+1:l+n2+1].max() > df1.high[l] ):
        return 0
    
    candle_body = abs(df1.open[l]-df1.close[l])
    upper_wick = df1.high[l]-max(df1.open[l], df1.close[l])
    if (upper_wick > candle_body) and (upper_wick > wick_threshold) :
        return 1

    return 0

In [18]:
def closeResistance(l,levels,lim, df):
    if len(levels)==0:
        return 0
    c1 = abs(df.high[l]-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    c2 = abs(max(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.high[l])))<=lim
    c3 = min(df.open[l],df.close[l])<min(levels, key=lambda x:abs(x-df.high[l]))
    c4 = df.low[l]<min(levels, key=lambda x:abs(x-df.high[l]))
    if( (c1 or c2) and c3 and c4 ):
        return min(levels, key=lambda x:abs(x-df.high[l]))
    else:
        return 0
    
def closeSupport(l,levels,lim, df):
    if len(levels)==0:
        return 0
    c1 = abs(df.low[l]-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c2 = abs(min(df.open[l],df.close[l])-min(levels, key=lambda x:abs(x-df.low[l])))<=lim
    c3 = max(df.open[l],df.close[l])>min(levels, key=lambda x:abs(x-df.low[l]))
    c4 = df.high[l]>min(levels, key=lambda x:abs(x-df.low[l]))
    if( (c1 or c2) and c3 and c4 ):
        return min(levels, key=lambda x:abs(x-df.low[l]))
    else:
        return 0

In [19]:
def is_below_resistance(l, level_backCandles, level, df):
    return df.loc[l-level_backCandles:l-1, 'high'].max() < level

def is_above_support(l, level_backCandles, level, df):
    return df.loc[l-level_backCandles:l-1, 'low'].min() > level        

In [20]:
def check_candle_signal(l, n1, n2, backCandles, df):
    ss = []
    rr = []
    for subrow in range(l-backCandles, l-n2):
        if support(df, subrow, n1, n2):
            ss.append(df.low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.high[subrow])
    
    ss.sort() #keep lowest support when popping a level
    for i in range(1,len(ss)):
        if(i>=len(ss)):
            break
        if abs(ss[i]-ss[i-1])<=0.0001: # merging close distance levels
            ss.pop(i)

    rr.sort(reverse=True) # keep highest resistance when popping one
    for i in range(1,len(rr)):
        if(i>=len(rr)):
            break
        if abs(rr[i]-rr[i-1])<=0.0001: # merging close distance levels
            rr.pop(i)

    #----------------------------------------------------------------------
    # joined levels
    rrss = rr+ss
    rrss.sort()
    for i in range(1,len(rrss)):
        if(i>=len(rrss)):
            break
        if abs(rrss[i]-rrss[i-1])<=0.0001: # merging close distance levels
            rrss.pop(i)
    cR = closeResistance(l, rrss, 150e-5, df)
    cS = closeSupport(l, rrss, 150e-5, df)
    #----------------------------------------------------------------------

    # cR = closeResistance(l, rr, 150e-5, df)
    # cS = closeSupport(l, ss, 150e-5, df)
    # could we consider the average RSI for the trend momentum?
    if (cR and is_below_resistance(l,6,cR, df) and df.RSI[l-1:l].min()<45 ):#and df.RSI[l]>65
        return 1
    elif(cS and is_above_support(l,6,cS,df) and df.RSI[l-1:l].max()>55 ):#and df.RSI[l]<35
        return 2
    else:
        return 0



In [21]:

n1 = 8
n2 = 6
backCandles = 140

signal = [0 for i in range(len(df))]

for row in tqdm(range(backCandles+n1, len(df)-n2)):
    signal[row] = check_candle_signal(row, n1, n2, backCandles, df)

df["signal"] = signal


100%|████████████████████████████████████████████████████████████████████████████| 19846/19846 [06:48<00:00, 48.55it/s]


In [22]:
df[df['signal']==1].count()

time           1
open           1
high           1
low            1
close          1
tick_volume    1
RSI            1
signal         1
dtype: int64

In [23]:
df[ (df['signal']==1) | (df['signal']==2)]

,time,open,high,low,close,tick_volume,RSI,signal
1626,2017-07-15 01:00:00,2165.435,2190.001,2136.0,2145.002,2400,34.280645,1


In [24]:

def pointpos(x):
    if x['signal']==1:
        return x['high']+1e-4
    elif x['signal']==2:
        return x['low']-1e-4
    else:
        return np.nan

df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [27]:
dfpl = df[100:300]

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.update_layout(
    autosize=False,
    width=1000,
    height=800, 
    paper_bgcolor='white',
    plot_bgcolor='white')
fig.update_xaxes(gridcolor='black')
fig.update_yaxes(gridcolor='black')
fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=8, color="blue"),
                name="Signal")
fig.show()

In [35]:
def check_candle_signal_plot(l, n1, n2, backCandles, df, proximity):
    ss = []
    rr = []
    for subrow in range(l-backCandles, l-n2):
        if support(df, subrow, n1, n2):
            ss.append(df.low[subrow])
        if resistance(df, subrow, n1, n2):
            rr.append(df.high[subrow])
    
    ss.sort() #keep lowest support when popping a level
    i = 0
    while i < len(ss)-1:
        if abs(ss[i]-ss[i+1]) <= proximity:
            # ss[i] = (ss[i]+ss[i+1])/2
            # del ss[i+1]
            del ss[i+1]
        else:
            i+=1

    rr.sort(reverse=True) # keep highest resistance when popping one
    i = 0
    while i < len(rr)-1:
        if abs(rr[i]-rr[i+1]) <= proximity:
            #rr[i] = (rr[i]+rr[i+1])/2
            #del rr[i+1]
            del rr[i]
        else:
            i+=1

    dfpl=df[l-backCandles-n1:l+n2+50]
    fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

    c=0
    while (1):
        if(c>len(ss)-1 ):
            break
        fig.add_shape(type='line', x0=l-backCandles-n1, y0=ss[c],
                    x1=l,
                    y1=ss[c],
                    line=dict(color="MediumPurple",width=2), name="Support"
                    )
        c+=1

    c=0
    while (1):
        if(c>len(rr)-1 ):
            break
        fig.add_shape(type='line', x0=l-backCandles-n1, y0=rr[c],
                    x1=l,
                    y1=rr[c],
                    line=dict(color="Red",width=2), name="Resistance"
                    )
        c+=1    

    # fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
    #             marker=dict(size=5, color="MediumPurple"),
    #             name="Signal")

    fig.update_layout(
    autosize=False,
    width=1000,
    height=800,)
    
    fig.show()
 
 
    #----------------------------------------------------------------------
    cR = closeResistance(l, rr, 150e-5, dfpl)
    cS = closeSupport(l, ss, 150e-5, dfpl)
    #print(cR, is_below_resistance(l,6,cR, dfpl))
    if (cR and is_below_resistance(l,6,cR, dfpl) ):#and df.RSI[l]>65
        return 1
    elif(cS and is_above_support(l,6,cS,dfpl) ):#and df.RSI[l]<35
        return 2
    else:
        return 0

In [50]:
check_candle_signal_plot(1900, 5,5,150,df,0.5)

0